In [1]:
from main import load_data,get_data,regress,regress_h

In example 8.4,we discuss the heteroscedastic problem.

Book give a estimated equation.


$$
\begin{array}{l}
\begin{array}{lllll}
\widehat{price}= & -21.77+ & 0.00207lotsize+ & 0.123sqrft+ & 13.85bdrms \\
            & (29.48) & (0.00064)     & (0.013)   & (9.01) \\
\end{array}\\
n=88,R^2=0.672
\end{array}\tag{8.17}
$$

(I write the formula in a long time.MathJax is confused with surplus \n input.)

We can use a basic command to do it.

In [2]:
r=regress('price~lotsize+sqrft+bdrms','hprice1')

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.672
Model:                            OLS   Adj. R-squared:                  0.661
Method:                 Least Squares   F-statistic:                     57.46
Date:                Sat, 30 Jan 2016   Prob (F-statistic):           2.70e-20
Time:                        15:41:45   Log-Likelihood:                -482.88
No. Observations:                  88   AIC:                             973.8
Df Residuals:                      84   BIC:                             983.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept    -21.7703     29.475     -0.739      0.4

Looks great.But its flexibility is lack. For $R^2_{\hat{u}^2}$ (what fuck the MathJax display ) , we need return low level to do that.

In [3]:
import statsmodels.api as sm

y_head='price'
X_head=['lotsize','sqrft','bdrms']
gd=get_data(y_head,X_head,'hprice1')
gd['resid']=r.resid
mod=sm.OLS(gd['resid']**2,sm.add_constant(gd[X_head]))
res=mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  resid   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.130
Method:                 Least Squares   F-statistic:                     5.339
Date:                Sat, 30 Jan 2016   Prob (F-statistic):            0.00205
Time:                        15:41:45   Log-Likelihood:                -896.99
No. Observations:                  88   AIC:                             1802.
Df Residuals:                      84   BIC:                             1812.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const      -5522.7948   3259.478     -1.694      0.094      -1.2e+04   959.035
lotsize        0.2015      0.071      2.838      0.006         0.060     0.343
sqrft          1.6910      1.464      1.155      0.251        -1.220     4.602
bdrms       1041.7602    996.381      1.046      0.299      -939.653  3023.173
==============================================================================
Omnibus:                      115.167   Durbin-Watson:                   2.351
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2267.859
Skew:                           4.381   Prob(JB):                         0.00
Kurtosis:                      26.276   Cond. No.                     6.41e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.41e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

So $R^2_{\hat{u}^2}=0.160$ (note you should call sm.add_constant, if not you will get constant-free model and a high $R^2$) .It is correspond with book output.

Then,we can go the $F$ test path or $\chi^2$ test path.

## Breusch-Pagan test for heteroskedasticity,BP test

if $H_0$ is true,then:

let $R^2_u=R^2_{\hat{u}^2}$ avoid the ugly symbol

LM statistic

$$ LM=nR^2_u \sim \chi^2_{k} $$

F statistic

$$ F=\frac{R^2_u/k}{(1-R^2_u)/(n-k-1)} \sim F_{k,n-k-1} $$


In [4]:
import scipy.stats as stats

k=res.model.df_model #3
n=res.model.nobs #88
R2=res.rsquared

#chi^2 test
LM=n*R2
print 'LM p-value: ',1-stats.chi2(k).cdf(LM)

#F test
F=(R2/k)/((1-R2)/(n-k-1))
print 'F p-value: ',1-stats.f(k,n-k-1).cdf(F)

LM p-value:  0.00278205954233
F p-value:  0.00204774440967


However,It's impossible a statistic model package don't contain this function.

In [5]:
from statsmodels.stats.diagnostic import het_breushpagan

print het_breushpagan(gd['resid'],sm.add_constant(gd[X_head])) # you don't need transform it to **2
#print het_breushpagan.__doc__

(14.092385514591086, 0.0027820595423349564, 5.338919367860953, 0.0020477444096706544)


They're explained by document string below.

    lm : float
        lagrange multiplier statistic
    lm_pvalue :float
        p-value of lagrange multiplier test
    fvalue : float
        f-statistic of the hypothesis that the error variance does not depend
        on x
    f_pvalue : float
        p-value for the f-statistic
        
The result is validated at all.There's a log trasform example too.

In [6]:
r2=regress('np.log(price)~np.log(lotsize)+np.log(sqrft)+bdrms','hprice1',summary=False)
LM,LM_p,F,F_p=het_breushpagan(r2.resid,r2.model.exog)
print 'LM p-value: ',LM_p
print 'F p-value: ',F_p

LM p-value:  0.238344821018
F p-value:  0.245145655823


Example 8.4 All done. Empirical study is easy!